In [41]:
%pip install numpy


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
from pyspark.sql import SparkSession

In [33]:
#spark = SparkSession.builder \
    #.appName("Read MovieLens from HDFS") \
    #.getOrCreate()

#spark = SparkSession.builder \
    #.appName("ALS Model") \
    #.config("spark.driver.memory", "4g") \
    #.config("spark.executor.memory", "4g") \
   # .getOrCreate()

#.config("spark.hadoop.fs.defaultFS", "hdfs://bigdata-node:8088") \
spark = SparkSession.builder \
    .appName("MovieLensAnalysis") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .config("spark.sql.shuffle.partitions", "20") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.3") \
    .getOrCreate()

Lire les fichiers

In [34]:
ratings_df = spark.read.option("header", True).csv("hdfs:///data/rating.csv")
movies_df = spark.read.option("header", True).csv("hdfs:///data/movie.csv")

Supprimer les doublons

In [35]:
ratings_df = ratings_df.dropDuplicates()
movies_df = movies_df.dropDuplicates()

Supprimer les valeurs nulles

In [36]:
ratings_df = ratings_df.dropna(subset=["userId", "movieId", "rating"])
movies_df = movies_df.dropna(subset=["movieId", "title"])

In [38]:
ratings_df.show(5)

25/05/05 13:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 13:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 13:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 13:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 13:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 13:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 13:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 13:53:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 13:53:36 WARN RowBasedKeyValueBatch: Calling spill() on

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|    593|   3.5|2005-04-02 23:31:01|
|     1|   1246|   3.5|2004-09-10 03:09:19|
|     1|   1350|   3.5|2004-09-10 03:15:58|
|     1|   1525|     3|2005-04-02 23:55:50|
|     1|   2959|     4|2004-09-10 03:08:18|
+------+-------+------+-------------------+
only showing top 5 rows



Affichage des données

In [39]:
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      6|         Heat (1995)|Action|Crime|Thri...|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     24|       Powder (1995)|        Drama|Sci-Fi|
|     31|Dangerous Minds (...|               Drama|
|     60|Indian in the Cup...|Adventure|Childre...|
+-------+--------------------+--------------------+
only showing top 5 rows



In [20]:
df_data = ratings_df.join(
    movies_df.select("movieId", "title"),
    on="movieId",
    how="left"
)

In [21]:
df_data.show(5)

+-------+------+------+-------------------+--------------------+
|movieId|userId|rating|          timestamp|               title|
+-------+------+------+-------------------+--------------------+
|      2|     1|   3.5|2005-04-02 23:53:47|      Jumanji (1995)|
|     29|     1|   3.5|2005-04-02 23:31:16|City of Lost Chil...|
|     32|     1|   3.5|2005-04-02 23:33:39|Twelve Monkeys (a...|
|     47|     1|   3.5|2005-04-02 23:32:07|Seven (a.k.a. Se7...|
|     50|     1|   3.5|2005-04-02 23:29:40|Usual Suspects, T...|
+-------+------+------+-------------------+--------------------+
only showing top 5 rows



In [22]:
df_data = df_data.withColumnsRenamed({
    'userId': 'user',
    'movieId': 'movie',
})

In [23]:
df_data.show(5)

+-----+----+------+-------------------+--------------------+
|movie|user|rating|          timestamp|               title|
+-----+----+------+-------------------+--------------------+
|    2|   1|   3.5|2005-04-02 23:53:47|      Jumanji (1995)|
|   29|   1|   3.5|2005-04-02 23:31:16|City of Lost Chil...|
|   32|   1|   3.5|2005-04-02 23:33:39|Twelve Monkeys (a...|
|   47|   1|   3.5|2005-04-02 23:32:07|Seven (a.k.a. Se7...|
|   50|   1|   3.5|2005-04-02 23:29:40|Usual Suspects, T...|
+-----+----+------+-------------------+--------------------+
only showing top 5 rows



In [24]:
from pyspark.sql.functions import col

df_data = df_data \
    .withColumn("user", col("user").cast("int")) \
    .withColumn("movie", col("movie").cast("int")) \
    .withColumn("rating", col("rating").cast("float")) \
    .withColumn("timestamp", col("timestamp").cast("int")) \
    .withColumn("title", col("title").cast("string"))

In [44]:
df_data_train, df_data_test = df_data.randomSplit([0.8, 0.2], 42)

In [30]:
# import ALS model from spark
from pyspark.ml.recommendation import ALS

# initialise the model 
als = ALS(maxIter=10, regParam=0.1, rank=10, userCol="user", itemCol="movie", 
          ratingCol="rating", coldStartStrategy="drop", nonnegative=True)

# train model 
als_model = als.fit(df_data_train)

In [31]:
dfs_preds = als_model.transform(df_data_test)

In [32]:
dfs_preds.show(5)

[Stage 255:=====================================>                  (8 + 4) / 12]

+-----+----+------+---------+--------------------+----------+
|movie|user|rating|timestamp|               title|prediction|
+-----+----+------+---------+--------------------+----------+
|   47|  95|   3.0|     NULL|Seven (a.k.a. Se7...| 3.0762284|
|  262|  95|   3.0|     NULL|Little Princess, ...| 3.4373078|
|  446|  95|   3.0|     NULL|Farewell My Concu...| 3.6016524|
|  592|  95|   3.0|     NULL|       Batman (1989)|  2.502191|
|  708|  95|   4.0|     NULL|Truth About Cats ...|  3.108661|
+-----+----+------+---------+--------------------+----------+
only showing top 5 rows



In [33]:
#als_model.save("hdfs:///data/als_prediction_rating")

In [26]:
from pyspark.ml.recommendation import ALSModel

als_model_loaded = ALSModel.load("hdfs:///data/als_prediction_rating")

In [47]:
dfs_preds_loaded = als_model_loaded.transform(df_data_test)

In [ ]:
dfs_preds_loaded.show(5)

In [46]:
df_data_test.show()

[Stage 36:>                                                         (0 + 1) / 1]

+-----+----+------+---------+----------------+
|movie|user|rating|timestamp|           title|
+-----+----+------+---------+----------------+
|    1|   8|   4.0|     NULL|Toy Story (1995)|
|    1|  13|   4.0|     NULL|Toy Story (1995)|
|    1|  16|   3.0|     NULL|Toy Story (1995)|
|    1|  31|   3.0|     NULL|Toy Story (1995)|
|    1|  58|   5.0|     NULL|Toy Story (1995)|
|    1|  80|   3.0|     NULL|Toy Story (1995)|
|    1|  96|   3.5|     NULL|Toy Story (1995)|
|    1| 114|   3.0|     NULL|Toy Story (1995)|
|    1| 134|   4.0|     NULL|Toy Story (1995)|
|    1| 135|   4.0|     NULL|Toy Story (1995)|
|    1| 136|   5.0|     NULL|Toy Story (1995)|
|    1| 140|   4.0|     NULL|Toy Story (1995)|
|    1| 144|   3.0|     NULL|Toy Story (1995)|
|    1| 155|   2.5|     NULL|Toy Story (1995)|
|    1| 172|   5.0|     NULL|Toy Story (1995)|
|    1| 204|   5.0|     NULL|Toy Story (1995)|
|    1| 232|   3.0|     NULL|Toy Story (1995)|
|    1| 296|   5.0|     NULL|Toy Story (1995)|
|    1| 318| 

In [48]:
from pyspark.ml.evaluation import RegressionEvaluator

# initiate evaluator
eval = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
# calculate rmse
rmse = eval.evaluate(dfs_preds_loaded)

# reset metric and evaluate mse, R2, mae & explained variance
r2 = eval.evaluate(dfs_preds_loaded, {eval.metricName: 'r2'})
mae = eval.evaluate(dfs_preds_loaded, {eval.metricName: 'mae'})
var = eval.evaluate(dfs_preds_loaded, {eval.metricName: 'var'})


print(
    "RMSE score = {}".format(rmse),
    "R2 score = {}".format(r2),
    "MAE score = {}".format(mae),
    "Explained variance score = {}".format(var),
    sep="\n"
)

[Stage 55:=======================>                                 (5 + 7) / 12]

RMSE score = 0.8157736828362929
R2 score = 0.39869052585871656
MAE score = 0.6375367684273177
Explained variance score = 0.4086564056899666


In [27]:
from pyspark.sql.functions import col

# ID de l'utilisateur
user_id = 29

# Créer un DataFrame avec l'utilisateur
user_df = spark.createDataFrame([(user_id,)], ["user"])

# Créer un DataFrame avec tous les films
all_movies_df = movies_df.select(col("movieId").cast("int").alias("movie"))

# Faire un produit cartésien pour obtenir (user, movie)
user_movies_df = user_df.crossJoin(all_movies_df)

# Prédire les notes
predicted_ratings_df = als_model_loaded.transform(user_movies_df)

# ✅ Préparer les titres des films sans ambiguïté
movies_renamed_df = movies_df.select(
    col("movieId").cast("int").alias("movie_id_for_join"),
    "title"
)

# ✅ Join propre entre prédictions et titres
result_df = predicted_ratings_df.join(
    movies_renamed_df,
    predicted_ratings_df.movie == movies_renamed_df.movie_id_for_join,
    how="left"
)

# Afficher les 10 meilleures prédictions
result_df.select("user", "movie", "title", "prediction") \
         .orderBy(col("prediction").desc()) \
         .show(100, truncate=False)

#result_df.select("user", "movie", "title", "prediction") \
#         .orderBy(col("prediction").desc()) \
#         .show(truncate=False, n=result_df.count())


[Stage 16:===================================================>    (11 + 1) / 12]

+----+------+------------------------------------------------------------------------------------------------------+----------+
|user|movie |title                                                                                                 |prediction|
+----+------+------------------------------------------------------------------------------------------------------+----------+
|29  |128091|Craig Ferguson: A Wee Bit o' Revolution (2009)                                                        |6.5495496 |
|29  |121029|No Distance Left to Run (2010)                                                                        |5.922186  |
|29  |77736 |Crazy Stone (Fengkuang de shitou) (2006)                                                              |5.7918434 |
|29  |129536|Code Name Coq Rouge (1989)                                                                            |5.3435593 |
|29  |99243 |Parasites, Les (1999)                                                                      

In [6]:
%pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pymongo]m1/2 [pymongo]

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
from pymongo import MongoClient

pdf = result_df.select("user", "movie", "title", "prediction") \
               .orderBy(col("prediction").desc()) \
               .limit(100) \
               .toPandas()

client = MongoClient("mongodb://mongodb:27017/")
collection = client["reco_db"]["predictions"]

collection.insert_many(pdf.to_dict(orient="records"))

print("✅ Données insérées dans MongoDB depuis result_df !")

✅ Données insérées dans MongoDB depuis result_df !


In [30]:
for doc in collection.find({"user": 29}):
    print(doc)

{'_id': ObjectId('6818ba8c4c93bd75c4aae51c'), 'user': 29, 'movie': 128091, 'title': "Craig Ferguson: A Wee Bit o' Revolution (2009)", 'prediction': 6.549549579620361}
{'_id': ObjectId('6818ba8c4c93bd75c4aae51d'), 'user': 29, 'movie': 121029, 'title': 'No Distance Left to Run (2010)', 'prediction': 5.922185897827148}
{'_id': ObjectId('6818ba8c4c93bd75c4aae51e'), 'user': 29, 'movie': 77736, 'title': 'Crazy Stone (Fengkuang de shitou) (2006)', 'prediction': 5.791843414306641}
{'_id': ObjectId('6818ba8c4c93bd75c4aae51f'), 'user': 29, 'movie': 129536, 'title': 'Code Name Coq Rouge (1989)', 'prediction': 5.343559265136719}
{'_id': ObjectId('6818ba8c4c93bd75c4aae520'), 'user': 29, 'movie': 99243, 'title': 'Parasites, Les (1999)', 'prediction': 5.145484447479248}
{'_id': ObjectId('6818ba8c4c93bd75c4aae521'), 'user': 29, 'movie': 102119, 'title': 'Yesterday Was a Lie (2008)', 'prediction': 5.13187837600708}
{'_id': ObjectId('6818ba8c4c93bd75c4aae522'), 'user': 29, 'movie': 98275, 'title': 'Octo

In [4]:
spark = SparkSession.builder \
    .appName("WriteToMongoDB") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.1.1") \
    .config("spark.mongodb.write.connection.uri", "mongodb://mongodb:27017/reco_db.predictions") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1850713e-7cbf-439b-b86f-13badd79f9d5;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.1.1 in central
	found org.mongodb#mongodb-driver-sync;4.8.2 in central
	[4.8.2] org.mongodb#mongodb-driver-sync;[4.8.1,4.8.99)
	found org.mongodb#bson;4.8.2 in central
	found org.mongodb#mongodb-driver-core;4.8.2 in central
	found org.mongodb#bson-record-codec;4.8.2 in central
downloading https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/10.1.1/mongo-spark-connector_2.12-10.1.1.jar ...
	[SUCCESSFUL ] org.mongodb.spark#mongo-spark-connector_2.12;10.1.1!mongo-spark-connector_2.12.jar (39ms)
downloading https://repo1.maven.org/maven2/org/mongodb/mongodb-driver-sync/4.8.2/mongodb-driver-sync-4.8.2.jar ...
	[SUCCESSFUL ] org.m

In [5]:
from pyspark.sql.functions import col

# Exemple de DataFrame à insérer (remplace par result_cleaned_df)
data = [(29, 1, "Film 1", 4.5), (29, 2, "Film 2", 4.0)]
columns = ["user", "movie", "title", "prediction"]
df = spark.createDataFrame(data, columns)

# Écriture dans MongoDB (base: reco_db, collection: predictions)
df.write \
    .format("mongodb") \
    .mode("overwrite") \
    .option("database", "reco_db") \
    .option("collection", "predictions") \
    .option("uri", "mongodb://mongodb:27017/") \
    .save()

25/05/05 13:04:43 ERROR Executor: Exception in task 7.0 in stage 0.0 (TID 7) 12]
java.lang.NoSuchMethodError: 'org.apache.spark.sql.catalyst.encoders.ExpressionEncoder org.apache.spark.sql.catalyst.encoders.RowEncoder$.apply(org.apache.spark.sql.types.StructType)'
	at com.mongodb.spark.sql.connector.schema.InternalRowToRowFunction.<init>(InternalRowToRowFunction.java:46)
	at com.mongodb.spark.sql.connector.schema.RowToBsonDocumentConverter.<init>(RowToBsonDocumentConverter.java:80)
	at com.mongodb.spark.sql.connector.write.MongoDataWriter.<init>(MongoDataWriter.java:78)
	at com.mongodb.spark.sql.connector.write.MongoDataWriterFactory.createWriter(MongoDataWriterFactory.java:54)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run(WriteToDataSourceV2Exec.scala:436)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run$(WriteToDataSourceV2Exec.scala:425)
	at org.apache.spark.sql.execution.datasources.v2.DataWritingSparkTask$.run(WriteToDataSourceV2Exec.

Py4JJavaError: An error occurred while calling o55.save.
: org.apache.spark.SparkException: Writing job failed.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobFailedError(QueryExecutionErrors.scala:902)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:411)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:359)
	at org.apache.spark.sql.execution.datasources.v2.OverwriteByExpressionExec.writeWithV2(WriteToDataSourceV2Exec.scala:243)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run(WriteToDataSourceV2Exec.scala:337)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run$(WriteToDataSourceV2Exec.scala:336)
	at org.apache.spark.sql.execution.datasources.v2.OverwriteByExpressionExec.run(WriteToDataSourceV2Exec.scala:243)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2

In [49]:
# Nettoyage : enlever les valeurs NaN (optionnel mais recommandé)
result_df.select("user", "movie", "title", "prediction") \
         .orderBy(col("prediction").desc()) \
         .show(30, truncate=False)

# Écriture dans MongoDB
result_df.write \
    .format("mongodb") \
    .mode("overwrite") \
    .option("database", "reco_db") \
    .option("collection", "predictions") \
    .option("uri", "mongodb://localhost:27017/") \
    .save()

+----+------+------------------------------------------------------------------------------------------------------+----------+
|user|movie |title                                                                                                 |prediction|
+----+------+------------------------------------------------------------------------------------------------------+----------+
|29  |128091|Craig Ferguson: A Wee Bit o' Revolution (2009)                                                        |6.5495496 |
|29  |121029|No Distance Left to Run (2010)                                                                        |5.922186  |
|29  |77736 |Crazy Stone (Fengkuang de shitou) (2006)                                                              |5.7918434 |
|29  |129536|Code Name Coq Rouge (1989)                                                                            |5.3435593 |
|29  |99243 |Parasites, Les (1999)                                                                      

Py4JJavaError: An error occurred while calling o870.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: mongodb. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:248)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.in